In [24]:
import pandas as pd
import numpy as np
import geopandas as gp

pd.set_option('display.max_columns',None)
shape = gp.read_file('.\\zones\\zones.shp').to_crs('EPSG:4326')
shape.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."


In [15]:
station_NYC = pd.read_csv('.\\m_station_NYC.csv', usecols=['start_station_name','start_lat','start_lng'])
geo_station_NYC = gp.GeoDataFrame(station_NYC, geometry=gp.points_from_xy(station_NYC['start_lng'], station_NYC['start_lat']), crs='EPSG:4326')

station_NYC = gp.sjoin(geo_station_NYC, shape, how='left')
station_NYC

,start_station_name,start_lat,start_lng,geometry,index_right,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough
0,Central Park West & W 76 St,40.778968,-73.973747,POINT (-73.97375 40.77897),42.0,43.0,0.099739,0.000380,Central Park,43.0,Manhattan
1,Pershing Square North,40.751873,-73.977706,POINT (-73.97771 40.75187),169.0,170.0,0.045769,0.000074,Murray Hill,170.0,Manhattan
2,Amsterdam Ave & W 82 St,40.785247,-73.976673,POINT (-73.97667 40.78525),238.0,239.0,0.063626,0.000205,Upper West Side South,239.0,Manhattan
3,1 Ave & E 16 St,40.732219,-73.981656,POINT (-73.98166 40.73222),223.0,224.0,0.044020,0.000098,Stuy Town/Peter Cooper Village,224.0,Manhattan
4,Great Jones St,40.727434,-73.993790,POINT (-73.99379 40.72743),113.0,114.0,0.031727,0.000047,Greenwich Village South,114.0,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...
3131,Grand Concourse & E 144 St,40.816870,-73.927985,POINT (-73.92799 40.81687),167.0,168.0,0.114517,0.000547,Mott Haven/Port Morris,168.0,Bronx
3132,Vesey St & Church St,40.712220,-74.010472,POINT (-74.01047 40.71222),260.0,261.0,0.027120,0.000034,World Trade Center,261.0,Manhattan
3133,E 39 St & 2 Ave,40.748033,-73.973828,POINT (-73.97383 40.74803),232.0,233.0,0.048036,0.000116,UN/Turtle Bay South,233.0,Manhattan
3134,E 91 St & 2 Ave,40.781152,-73.949630,POINT (-73.94963 40.78115),262.0,263.0,0.037017,0.000066,Yorkville West,263.0,Manhattan


In [16]:
station_NYC.drop(['geometry','index_right','OBJECTID','Shape_Leng','Shape_Area','zone','borough'], axis=1, inplace=True)
station_NYC.rename(columns={'start_station_name':'station_name', 'start_lat':'station_lat', 'start_lng':'station_lng', 'LocationID':'zone_id'}, inplace=True)
station_NYC.head()

,station_name,station_lat,station_lng,zone_id
0,Central Park West & W 76 St,40.778968,-73.973747,43.0
1,Pershing Square North,40.751873,-73.977706,170.0
2,Amsterdam Ave & W 82 St,40.785247,-73.976673,239.0
3,1 Ave & E 16 St,40.732219,-73.981656,224.0
4,Great Jones St,40.727434,-73.993790,114.0


In [17]:
station_NYC.to_csv('.\\m_station_NYC.csv')

In [26]:
zone_NYC = shape.drop(['geometry','OBJECTID'], axis=1)
zone_NYC.rename(columns={'Shape_Leng':'zone_leng', 'Shape_Area':'zone_area', 'zone':'zone_area', 'borough':'zone_borough', 'LocationID':'zone_id'}, inplace=True)
zone_NYC.head()

,zone_leng,zone_area,zone_area,zone_id,zone_borough
0,0.116357,0.000782,Newark Airport,1,EWR
1,0.433470,0.004866,Jamaica Bay,2,Queens
2,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx
3,0.043567,0.000112,Alphabet City,4,Manhattan
4,0.092146,0.000498,Arden Heights,5,Staten Island


In [27]:
zone_NYC.to_csv('.\\zone_NYC.csv')

In [28]:
import pandas as pd 
import numpy as np
import datetime as d
import calendar as c 
from datetime import timedelta

station_data = pd.read_csv('.\\m_station_data.csv', usecols=['started_at','ended_at','start_station_name','start_lat','start_lng','end_station_name','end_lat','end_lng','start_id','end_id'])
station_data.head()

,started_at,ended_at,start_station_name,start_lat,start_lng,end_station_name,end_lat,end_lng,start_id,end_id
0,2019-01-01 00:01:47.4010,2019-01-01 00:07:07.5810,Central Park West & W 76 St,40.778968,-73.973747,W 89 St & Columbus Ave,40.788221,-73.970416,2494.0,1669.0
1,2019-01-01 00:04:43.7360,2019-01-01 00:10:00.6080,Pershing Square North,40.751873,-73.977706,E 39 St & 2 Ave,40.747804,-73.973442,1.0,3133.0
2,2019-01-01 00:06:03.9970,2019-01-01 00:15:55.4380,Amsterdam Ave & W 82 St,40.785247,-73.976673,E 77 St & 3 Ave,40.773142,-73.958562,2263.0,2285.0
3,2019-01-01 00:07:03.5450,2019-01-01 00:52:22.6500,1 Ave & E 16 St,40.732219,-73.981656,W 15 St & 6 Ave,40.738046,-73.996430,1803.0,1932.0
4,2019-01-01 00:07:35.9450,2019-01-01 00:12:39.5020,Great Jones St,40.727434,-73.993790,E 20 St & Park Ave,40.738274,-73.987520,1634.0,1940.0


In [29]:
zone_map = {station_NYC.loc[i,'station_name'] : station_NYC.loc[i, 'zone_id'] for i in range(station_NYC.shape[0])}
station_data['start_zone'] = station_data['start_station_name'].apply(lambda x: zone_map[x] if x in zone_map else np.NaN)
station_data['end_zone'] = station_data['end_station_name'].apply(lambda x: zone_map[x] if x in zone_map else np.NaN)

In [30]:
station_data['started_at'] = pd.to_datetime(station_data['started_at'], format='%Y-%m-%d %H:%M:%S')
station_data['ended_at'] = pd.to_datetime(station_data['ended_at'], format='%Y-%m-%d %H:%M:%S')
station_data.head()

,started_at,ended_at,start_station_name,start_lat,start_lng,end_station_name,end_lat,end_lng,start_id,end_id,start_zone,end_zone
0,2019-01-01 00:01:47.401,2019-01-01 00:07:07.581,Central Park West & W 76 St,40.778968,-73.973747,W 89 St & Columbus Ave,40.788221,-73.970416,2494.0,1669.0,43.0,238.0
1,2019-01-01 00:04:43.736,2019-01-01 00:10:00.608,Pershing Square North,40.751873,-73.977706,E 39 St & 2 Ave,40.747804,-73.973442,1.0,3133.0,170.0,233.0
2,2019-01-01 00:06:03.997,2019-01-01 00:15:55.438,Amsterdam Ave & W 82 St,40.785247,-73.976673,E 77 St & 3 Ave,40.773142,-73.958562,2263.0,2285.0,239.0,236.0
3,2019-01-01 00:07:03.545,2019-01-01 00:52:22.650,1 Ave & E 16 St,40.732219,-73.981656,W 15 St & 6 Ave,40.738046,-73.996430,1803.0,1932.0,224.0,90.0
4,2019-01-01 00:07:35.945,2019-01-01 00:12:39.502,Great Jones St,40.727434,-73.993790,E 20 St & Park Ave,40.738274,-73.987520,1634.0,1940.0,114.0,107.0


In [31]:
delta_incres = 60
current_time = d.datetime(2019, 1, 1)
end_time = d.datetime(2021, 9, 1)

result_station = []
while current_time <= end_time:
    day_num = c.monthrange(current_time.year, current_time.month)[1]
    result_station.append(np.zeros([(1440 // delta_incres * day_num), zone_NYC.shape[0], 2], dtype=int))
    current_time += timedelta(days=day_num)

In [33]:
for i in range(station_data.shape[0]):
    start_zone = station_data.loc[i, 'start_zone']
    end_zone = station_data.loc[i, 'end_zone']

    if (not np.isnan(start_zone)) and (not np.isnan(end_zone)):
        started_at = station_data.loc[i, 'started_at']
        s_0 = (started_at.year-2019)*12 + (started_at.month-1)
        s_1 = ((started_at.day-1)*24*60 + (started_at.hour)*60 + (started_at.minute)) // delta_incres
        result_station[s_0][s_1][int(start_zone)-1][0] += 1
    
        ended_at = station_data.loc[i, 'ended_at']
        e_0 = (ended_at.year-2019)*12 + (ended_at.month-1)
        e_1 = ((ended_at.day-1)*24*60 + (ended_at.hour)*60 + (ended_at.minute)) // delta_incres
        result_station[e_0][e_1][int(end_zone)-1][1] += 1

In [34]:
current_time = d.datetime(2019, 1, 1)
end_time = d.datetime(2021, 9, 1)

i = 0
while current_time <= end_time:
    day_num = c.monthrange(current_time.year, current_time.month)[1]
    np.save(('.\\processed_data\\{}{:02d}_zone.npy'.format(current_time.year, current_time.month)), result_station[i])
    current_time += timedelta(days=day_num)
    i += 1

In [36]:
station_data.to_csv('.\\m_station_data.csv')